In [ ]:
from inputs.constants import SUPPLEMENT_PATH
from emutools.tex import StandardTexDoc
from aust_covid.calibration import get_targets
from aust_covid.utils import add_image_to_doc
from aust_covid.plotting import plot_targets

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'targets', 'Target construction', 'austcovid')
targets = get_targets(app_doc)

In [ ]:
fig = plot_targets(targets)
add_image_to_doc(fig, 'targets', 'Calibration targets with raw data from which they were derived.', app_doc, 'Targets')
app_doc.write_doc()
fig